In [1]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup 
from IPython.display import Markdown, display, update_display
import requests 
from typing import List

In [2]:
import requests

response = requests.post(
    'http://localhost:11434/api/generate',
    json={
        "model": "llama3",
        "prompt": "Who is Hussain?",
        "stream": False
    }
)

# Print the full response to see what's inside
data = response.json()
print(data)


{'model': 'llama3', 'created_at': '2025-08-19T11:06:04.879216567Z', 'response': "Hussain (570-661 CE) was a revered Islamic saint, martyr, and grandson of the Prophet Muhammad. He played a crucial role in the early history of Islam and is considered one of the most important figures in Shia Islam.\n\nHussain ibn Ali was born in Medina, Arabia, to Imam Ali, the first Imam of Shia Islam, and Fatima, the daughter of the Prophet Muhammad. Hussain grew up in a family deeply rooted in Islamic values and became known for his piety, wisdom, and courage.\n\nIn 680 CE, Hussain led a caravan with his family and followers from Medina to Mecca, intending to perform the Hajj pilgrimage. However, he was confronted by the army of Yazid I, the Umayyad caliph who had seized power through treachery and usurpation. When Hussain refused to pledge allegiance to Yazid, a brutal battle ensued at Karbala, near modern-day Iraq.\n\nDespite being vastly outnumbered (around 72-100 men against an army of thousands)

In [73]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    url: str
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        
        self.title = soup.title.string if soup.title else None
        print(self.title if self.title else "No title found")
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
      
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"





In [74]:
ed = Website("https://edwarddonner.com")
ed.links

Home - Edward Donner


['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [125]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs or other relevant pages.\n"
link_system_prompt += "You are an assistant that MUST return ONLY valid JSON. No prose, no comments, no Markdown\
Return exactly this shape:,  as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
Rules:
- Output must be a single JSON object parsable by Python json.loads.
- Do not truncate URLs. If a relevant link doesn't exist, omit it (do NOT invent).
- Prefer About / Company / Careers / Blog / Contact / Social profiles relevant to a brochure.
"""

In [126]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs or other relevant pages.
You are an assistant that MUST return ONLY valid JSON. No prose, no comments, no MarkdownReturn exactly this shape:,  as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
Rules:
- Output must be a single JSON object parsable by Python json.loads.
- Do not truncate URLs. If a relevant link doesn't exist, omit it (do NOT invent).
- Prefer About / Company / Careers / Blog / Contact / Social profiles relevant to a brochure.



In [127]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


In [128]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

def messages_for(website):
    return [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(website)}
    ]
messages_for(ed)

In [131]:
from rich.console import Console
import requests
from rich.markdown import Markdown
console = Console()
import json 
import requests
import requests
import json
import re
from urllib.parse import urljoin

def get_links(url):
    website = Website(url)

    # Strongly worded system prompt helps; JSON mode enforces syntax.
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": [
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(website)}
            ],
            # 🔒 Force JSON-only output
            "format": "json",
            "stream": False,
            # Make the model deterministic/less chatty
            "options": {"temperature": 0}
        },
      
    )

    response.raise_for_status()
    data = response.json()

    # In JSON mode, this should already be a clean JSON string
    text = data["message"]["content"].strip()

    # Some UIs wrap JSON in ```json ... ```; strip if present
    m = re.search(r"```(?:json)?\s*(\{.*\})\s*```", text, re.DOTALL)
    if m:
        text = m.group(1)

    try:
        result = json.loads(text)
    except json.JSONDecodeError as e:
        # Helpful debug so you can see exactly what the model returned
        raise ValueError(f"Model did not return valid JSON.\nError: {e}\nRaw text:\n{text}") from e

    # Validate expected shape
    if not isinstance(result, dict) or "links" not in result or not isinstance(result["links"], list):
        raise ValueError(f"Unexpected shape from model: {result}")

    # Normalize/clean items
    cleaned = {"links": []}
    for item in result["links"]:
        if not isinstance(item, dict):
            continue
        link_type = item.get("type")
        link_url = item.get("url")
        if not isinstance(link_type, str) or not isinstance(link_url, str):
            continue
        # Resolve relative URLs to absolute
        abs_url = urljoin(website.url, link_url)
        cleaned["links"].append({"type": link_type.strip(), "url": abs_url})

    return cleaned




In [132]:
get_links("https://edwarddonner.com")


Home - Edward Donner


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'careers page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

<h1>NOW BROCHURE</h1>

In [135]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


In [136]:
print(get_all_details("https://edwarddonner.com"))

Home - Edward Donner
Home - Edward Donner
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'}, {'type': 'social profile', 'url': 'https://www.facebook.com/edward.donner.52'}]}
About - Edward Donner
Home - Edward Donner
Ed Donner - Nebula.io | LinkedIn
No title found
Error
Landing page:
Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head

In [137]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [138]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


In [140]:
get_brochure_user_prompt("edward", "https://edwarddonner.com")

Home - Edward Donner
Home - Edward Donner
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'}, {'type': 'social profile', 'url': 'https://www.facebook.com/edward.donner.52'}]}
About - Edward Donner
Home - Edward Donner
Ed Donner - Nebula.io | LinkedIn
No title found
Error


'You are looking at a company called: edward\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previous

In [152]:
def messages_are(website):
     return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_brochure_user_prompt("edward", "https://edwarddonner.com")}
    ]

messages_are(ed)    

Home - Edward Donner
Home - Edward Donner
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'}, {'type': 'social profile', 'url': 'https://www.facebook.com/edward.donner.52'}]}
About - Edward Donner
Home - Edward Donner
Ed Donner - Nebula.io | LinkedIn
No title found
Error


[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of several relevant pages from a company website and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.Include details of company culture, customers and careers/jobs if you have the information.'},
 {'role': 'user',
  'content': 'You are looking at a company called: edward\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losi

In [155]:
from rich.console import Console
import requests
from rich.markdown import Markdown
console = Console()

def summarize(company_name, url):
    website = Website(url)
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": messages_are(website),
            "stream": False
        }
    )
    data = response.json()
    return data['message']['content']




In [156]:
summary = summarize("edward", "https://edwarddonner.com")

Home - Edward Donner
Home - Edward Donner
Home - Edward Donner
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'}, {'type': 'social profile', 'url': 'https://www.facebook.com/edward.donner.52'}]}
About - Edward Donner
Home - Edward Donner
Ed Donner - Nebula.io | LinkedIn
No title found
Error


In [157]:
md = Markdown(summary)
console.print(md)

Edward Donner Brochure                                                                                             

Company Overview Edward Donner is a visionary company that leverages Generative AI and machine learning to         
revolutionize the recruitment industry. Founded by Ed, the co-founder and CTO of Nebula.io, the company's mission  
is to help people discover their potential and pursue their reason for being.                                      

The Problem 77% of people don't consider themselves inspired or engaged at work. Edward Donner aims to change this 
statistic by helping recruiters source, understand, engage, and manage talent more effectively.                    

The Solution Nebula.io's patented model matches people with roles with greater accuracy and speed than previously  
imaginable - no keywords required. This innovative approach enables recruiters to find the perfect candidates for  
their clients' needs.                                                                                              

Background Ed has a 20-year career in Financial Services, during which he founded AI startup untapt, which was     
acquired by GQR's parent company in 2021. He is passionate about applying AI to real-world problems and tackling   
tangible issues that affect everyone.                                                                              

Current Projects Edward Donner is currently focused on developing Nebula.io, a platform that helps recruiters find 
the right candidates for their clients' needs. Ed also shares his knowledge and expertise through online courses   
and workshops, where participants can learn about LLMs (Large Language Models) and how to become an expert in this 
field.                                                                                                             

Culture At Edward Donner, innovation, collaboration, and passion are highly valued. The company encourages         
networking and building meaningful connections with others who share similar interests and goals. Ed invites       
prospective customers, investors, and recruits to connect with him for a virtual or real coffee to discuss their   
shared vision.                                                                                                     

Contact Information                                                                                                

 • Email: ]8;id=655951;mailto:ed@edwarddonner.com\ed@edwarddonner.com]8;;\                                                                                      
 • Website: www.edwarddonner.com                                                                                   
 • Social Media: LinkedIn, Twitter, Facebook                                                                       
 • Newsletter: Subscribe by typing your email address